# 深層学習ノートブック-2 自動微分(Autograd)
* pytorchでは計算グラフを自動的に構築し，勾配を逆方向に計算する  
* tensor生成時にrequires_grad=Trueを設定することで自動微分を有効にする  
* 計算後に.backward()を呼ぶことで自動的に勾配が計算される  
* 勾配情報は.grad属性に累積される  
* 末端ノードに対する勾配しか保存されない
    * 中間ノードに対する勾配を保存したい場合は当該tensorに対して.retain_grad()を実行する  


In [10]:
# import torchでpytorchをimport
import torch

# torchで使う疑似乱数のseedは設定
torch.manual_seed(42)

In [17]:
# tensorを作成し、requires_grad=Trueを設定して演算を追跡
x = torch.ones(2, 2, requires_grad=True)

# tensorに対する捜査
y = x + 2
z = y * y * 3
out = z.mean()

# 勾配を計算
out.backward()

# 勾配d(out)/dxを出力
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


[自分用メモ書き]  
上記では数学で求めるような導関数の計算過程なしに微分係数が求まっているので、  
数学の感覚からはちょっとわかりづらかったが、やっていることは下記。  
数学的に計算すると確かに上記のようになる。  
（ただし、y * yの部分は行列積ではなくアダマール積（対応要素同士の積）であることに注意。）  

1. tensorのxを下記のようにおく  
    $\bm{X} = \left(
    \begin{matrix} 
    x_{1} & x_{2} \\ 
    x_{3} & x_{4} \\
    \end{matrix} 
    \right)$

2. すると、上記のy,z,outは下記のように書ける。  
    $\bm{Y} = \left(
    \begin{matrix} 
    x_{1}+2 & x_{2}+2 \\ 
    x_{3}+2 & x_{4}+2 \\
    \end{matrix} 
    \right)$  

    $\bm{Z} = \left(
    \begin{matrix} 
    3(x_{1}+2)^2 & 3(x_{2}+2)^2 \\ 
    3(x_{3}+2)^2 & 3(x_{4}+2)^2 \\
    \end{matrix} 
    \right)$

    $\bm{out} = \frac{3}{4}\sum_{i=1}^{n}(x_i + 2)^2 \quad $


3. 試しにoutを$x_1$で偏微分すると、  
    $\frac{\partial out }{\partial x_1} = \frac{3}{2}(x_1+2)$  

    この$x_1$に上記例の1を代入すると最終的な結果の4.5に一致する。  
    すなわち、x.gradの結果を数学的に見ると、  
    計算過程で導関数を求め、そこへ入力tensor(x)を代入し、各要素に対応する偏微分係数を返している模様。  
